In [1]:
#imports and scraping raw data from site
#df is the actual betting line
#df2 has other info, mostly using to get player id to name dict

import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import datetime as dt
import random
import re
from unidecode import unidecode


In [2]:
import urllib3
import json

import urllib
class AppURLopener(urllib.request.FancyURLopener):
    version = "Mozilla/5.0"

    
dog_props_url = 'https://api.underdogfantasy.com/beta/v3/over_under_lines'
opener = AppURLopener()
response = opener.open(dog_props_url)

print(response.status)

data = json.loads(response.read())

C:\Users\Shared.Westover\AppData\Local\Temp\ipykernel_12828\3263823425.py:10: DeprecationWarning: AppURLopener style of invoking requests is deprecated. Use newer urlopen functions/methods
  opener = AppURLopener()


200


In [3]:
data.keys()

dict_keys(['appearances', 'games', 'over_under_lines', 'players', 'solo_games'])

In [4]:
df = pd.json_normalize(list(data.values()))
df.index = data.keys()
new = df.T['over_under_lines'].values
lines = pd.json_normalize(new)
lines.sample(2)

,id,live_event,live_event_stat,options,over_under_id,rank,stat_value,status,over_under.id,over_under.appearance_stat.id,over_under.appearance_stat.appearance_id,over_under.appearance_stat.display_stat,over_under.appearance_stat.graded_by,over_under.appearance_stat.pickem_stat_id,over_under.appearance_stat.stat,over_under.boost,over_under.scoring_type_id,over_under.title
711,701e5d81-757b-4c02-9796-99cad56fa23c,False,None,[{'id': 'b95bae9f-523a-4454-ad9f-cc70593c6a42'...,d61ca9a9-d9d9-463d-9f0d-66489284000a,20199099,33.05,active,d61ca9a9-d9d9-463d-9f0d-66489284000a,dc8a8315-b6bd-41f6-bb79-a570e39103fe,83a02534-51cd-491e-a31d-f337f3b00166,Fantasy Points,high_score,54c0bded-86c9-42b0-9ff1-47927be5f500,fantasy_points,None,5903e326-17af-4e3b-963b-c142481b6014,Mark Hubbard Fantasy Points O/U
640,0fb7069e-02ea-4777-bb78-d3a86a1ce1f2,False,None,[{'id': 'a8a8a65d-58b0-4a66-9c40-f951e7e73f8b'...,26ce4463-58fa-4f6d-ad15-edbccb9fcdab,20199091,5.0,active,26ce4463-58fa-4f6d-ad15-edbccb9fcdab,012ea860-099f-4c5d-9a3e-f0371f6a1ea9,e5630527-85ca-484d-a0e5-4e3ce9e5c12b,Birdies or Better,high_score,97cff776-1653-4386-8ae7-6ce120f57bb0,birdies_or_better,None,None,J.J. Spaun Birdies or Better O/U


In [5]:
test =  pd.json_normalize(df.T['players'])
player_df = test.copy(deep=True)
player_df = player_df[['id','first_name', 'last_name',
       'sport_id','team_id']]
player_df = player_df.dropna()#

player_df.loc[player_df.sport_id=='ESPORTS']
x = player_df[player_df.sport_id=='ESPORTS']
p2esport = x[['last_name','first_name']].to_dict('split')
p2esport = dict(p2esport['data'])

In [6]:
test =  pd.json_normalize(df.T['players'])
player_df = test.copy(deep=True)
player_df = player_df[['id','first_name', 'last_name',
       'sport_id','team_id']]
player_df = player_df.dropna()
player_df.loc[player_df['sport_id'] != 'ESPORTS', 'full_name'] = player_df['first_name']+' ' + player_df['last_name']
player_df.loc[player_df['sport_id'] == 'ESPORTS', 'full_name'] = player_df['last_name']
#player_df['full_name'] = player_df['first_name']+' ' + player_df['last_name']
player_df = player_df[['full_name','sport_id','team_id']]
p2l = player_df[['full_name','sport_id']].to_dict('split')
p2tm = player_df[['full_name','team_id']].to_dict('split')
temp = p2l['data']
temp2 = p2tm['data']
p2l = dict(temp)
p2tm = dict(temp2)

In [7]:
player_df

,full_name,sport_id,team_id
0,Dean Kremer,MLB,1fd765d9-7354-4926-bb4b-368d86d202b0
1,Adley Rutschman,MLB,1fd765d9-7354-4926-bb4b-368d86d202b0
2,Gunnar Henderson,MLB,1fd765d9-7354-4926-bb4b-368d86d202b0
3,Anthony Santander,MLB,1fd765d9-7354-4926-bb4b-368d86d202b0
4,Ryan Mountcastle,MLB,1fd765d9-7354-4926-bb4b-368d86d202b0
...,...,...,...
539,C. Seager + M. Semien,MLBCOMBOS,976becd7-ee88-4db5-9ae6-d1bb6b2b16a8
540,C. Mullins + R. Mountcastle,MLBCOMBOS,a2ddbb25-4d77-4b86-9b27-27e150eede67
541,A. Garcia + N. Lowe,MLBCOMBOS,976becd7-ee88-4db5-9ae6-d1bb6b2b16a8
542,J. Heim + E. Carter,MLBCOMBOS,976becd7-ee88-4db5-9ae6-d1bb6b2b16a8


In [8]:
player_df.sport_id.unique()

array(['MLB', 'NHL', 'CFB', 'WNBA', 'NBA', 'NFL', 'ESPORTS', 'RACING',
       'SOCCER', 'NBASZN', 'NHLSZN', 'MLBCOMBOS'], dtype=object)

In [9]:
first2 = player_df['full_name'].apply(lambda x: ' '.join(x.split()[0:2]))
names = pd.DataFrame((first2,player_df.full_name))
names = names.T
names.columns=['first2','fullname']
names.loc[names.first2=='Jazz Chisholm']
first2full = names.to_dict('split')
first2full = dict(first2full['data'])

In [10]:
games = pd.json_normalize(df.T['games'].values).dropna(how='all')
#games = games.loc[games['sport_id']=='NBA']
games.tail(3)

,id,away_team_id,away_team_score,home_team_id,home_team_score,match_progress,period,scheduled_at,season_type,sport_id,status,title,type,year
40,53558.0,eeedd15f-6b08-4fb3-9e5d-07899af8a11a,0.0,eedeec41-ad6e-46ec-aa31-4c7cbd906c92,0.0,Wed 08:00am,0.0,2023-10-11T12:00:00Z,regular,ESPORTS,scheduled,Val: Attacking Soul Esports vs. Dragon Ranger ...,Game,2023.0
41,52352.0,c080cfb3-8185-4c88-b98d-f1cdb0f05a15,0.0,b3ef6f3e-a846-47da-924b-589b1879c968,0.0,Tue 05:00pm,0.0,2023-10-10T21:00:00Z,regular,ESPORTS,scheduled,CS:GO: BeBold vs. FURIA Academy,Game,2023.0
42,52354.0,7530af25-521f-4657-9641-af26a1a62541,0.0,ea45e149-84a2-4823-a98c-3eae3b2ca8e0,0.0,Tue 07:00pm,0.0,2023-10-10T23:00:00Z,regular,SOCCER,scheduled,Boca Juniors vs Belgrano,Game,2023.0


In [11]:
def to_central(date_str):
    x = pd.to_datetime(date_str)
    x = x.tz_convert('US/Central')
    return str(x.date())
to_central('2023-05-20T00:40:00Z')

'2023-05-19'

In [12]:
games.scheduled_at = games.scheduled_at.apply(to_central)

In [13]:
#grabbing team names from match title
matchup = games['title']
home = []
away = []
for i in matchup:
    temp = re.split(' @ |, |vs.',i)
    home.append(temp[0])
    away.append(temp[1])
#fixing esports team names
for i in range(len(home)):
    home[i] = home[i].split(': ')[-1]
    
for i in range(len(away)):
    away[i] = away[i].split(': ')[-1]

matchup = pd.DataFrame((away,home,games.away_team_id,games.home_team_id)).T
matchup.columns = ['Home', 'Away','away_id','home_id']
dates = games['scheduled_at'].values
#temp = []
#for i in dates:
#    temp.append(i.split('T')[0])
matchup['Date'] = dates
id2tm =  matchup[['home_id','Home']].to_dict('split')
id2tmaway =  matchup[['away_id','Away']].to_dict('split')
id2tm = dict(id2tm['data'])
id2tmaway = dict(id2tmaway['data'])
id2tm.update(id2tmaway)

In [14]:
games

,id,away_team_id,away_team_score,home_team_id,home_team_score,match_progress,period,scheduled_at,season_type,sport_id,status,title,type,year
0,52101.0,1fd765d9-7354-4926-bb4b-368d86d202b0,0.0,f85132d6-ce4f-461f-ab0d-97810d017c94,0.0,Tue 08:03pm,0.0,2023-10-10,post,MLB,scheduled,BAL @ TEX,Game,2023.0
1,52102.0,02c19f09-f45a-4808-a128-e5bbf9d9577e,0.0,25101c7a-340a-4c08-a93d-979242271c89,0.0,Wed 05:07pm,0.0,2023-10-11,post,MLB,scheduled,ATL @ PHI,Game,2023.0
2,52100.0,5a5a4e8a-d364-4fba-9b76-add9ed5d9ad7,0.0,0331354e-f6cb-4dba-ba55-7c5c40ab9f5e,0.0,Wed 09:07pm,0.0,2023-10-11,post,MLB,scheduled,LAD @ ARI,Game,2023.0
3,50342.0,e6fa00a2-55eb-45f3-b8de-2a7ad7329f30,0.0,5cdace81-efe8-4b20-ada3-e9ceb0d52048,0.0,Tue 05:34pm,0.0,2023-10-10,regular,NHL,scheduled,NSH @ TB,Game,2023.0
4,50343.0,3b4a9985-39fa-4e99-bb10-087408e2499a,0.0,89137af7-f2e1-4d68-aa58-a6e152831cec,0.0,Tue 08:04pm,0.0,2023-10-10,regular,NHL,scheduled,CHI @ PIT,Game,2023.0
5,50344.0,612c157d-112f-4aa9-a76b-3d17f6b99bb5,0.0,21c32c27-2668-4d04-a209-497292923e36,0.0,Tue 10:34pm,0.0,2023-10-10,regular,NHL,scheduled,SEA @ VGK,Game,2023.0
6,43365.0,6a701ae6-65f3-4787-8d98-529c32fb0090,0.0,6bc3b90b-9b1d-4ca8-ae24-e0ecf6328bbc,0.0,Tue 07:00pm,0.0,2023-10-10,regular,CFB,scheduled,LT @ MTS,Game,2023.0
7,43367.0,50647f25-01fe-46f2-a485-6ea1c0141ba4,0.0,c3d79b73-7451-487b-ae57-2ee127d88467,0.0,Tue 07:30pm,0.0,2023-10-10,regular,CFB,scheduled,LIB @ JVS,Game,2023.0
8,43366.0,9a9214e4-f0d9-4ab5-8ce6-cd252a294fcf,0.0,0ab8aa40-ab5c-4a9a-8910-13326a5db3b9,0.0,Tue 07:30pm,0.0,2023-10-10,regular,CFB,scheduled,CC @ APP,Game,2023.0
9,43368.0,da416f08-5993-4c02-bafd-58a831d90fb7,0.0,e881ba8d-d68a-40f0-a01c-68f6b6c2bb63,0.0,Wed 07:30pm,0.0,2023-10-11,regular,CFB,scheduled,TEP @ FIU,Game,2023.0


In [15]:
id2tm

{'f85132d6-ce4f-461f-ab0d-97810d017c94': 'TEX',
 '25101c7a-340a-4c08-a93d-979242271c89': 'PHI',
 '0331354e-f6cb-4dba-ba55-7c5c40ab9f5e': 'ARI',
 '5cdace81-efe8-4b20-ada3-e9ceb0d52048': 'TB',
 '89137af7-f2e1-4d68-aa58-a6e152831cec': 'PIT',
 '21c32c27-2668-4d04-a209-497292923e36': 'VGK',
 '6bc3b90b-9b1d-4ca8-ae24-e0ecf6328bbc': 'MTS',
 'c3d79b73-7451-487b-ae57-2ee127d88467': 'JVS',
 '0ab8aa40-ab5c-4a9a-8910-13326a5db3b9': 'APP',
 'e881ba8d-d68a-40f0-a01c-68f6b6c2bb63': 'FIU',
 '8f4df877-4dcc-453e-87be-e6adc503a0d9': 'NMS',
 'c8a52067-5c73-44dc-b4ef-ee106f408933': 'HOU',
 'ee63cf64-e034-4569-ac47-ea6252fdffe6': 'ECU',
 '421f09b9-43c3-422a-ac60-7ef2d69f326b': 'WIS',
 'ec4bf359-4c76-4e8d-b3f1-ba323966e51e': 'LVA',
 '04fc51d6-320c-4838-a087-f492067b71f5': 'ATL',
 '4ea17ce5-80e9-47b3-9955-2729a02bbe92': 'MIA',
 '09540cc3-fc27-4a59-b7a6-7a63f0e1df95': 'HOU',
 '56a5d8c8-eb72-47db-9e93-5b24d8f5443a': 'MEM',
 '40db32e9-eeb1-4fcf-9e0a-116518312ecf': 'NOP',
 'bd4b60cd-5adb-4cc6-8e8a-97084501f386': 

In [16]:
lines = lines[['stat_value','over_under.appearance_stat.stat', 'over_under.title']]
lines
players = lines['over_under.title'].values
guys = []
for i in players:
    temp = i.split()
    final = temp[0] + ' '+temp[1]
    guys.append(final)
    
guys
lines= lines.rename(columns={'stat_value':'Line','over_under.appearance_stat.stat':'Stat', 'over_under.title':'title'})
lines['Player']= guys
lines['Player'] = lines['Player'].apply(lambda x: unidecode(x).replace('_',' '))


In [17]:
def map_name(x):
    if x is None:
        return None
    if x in first2full:
        return first2full[x]
    elif x.split(': ')[-1] in first2full:
        return x.split(': ')[-1]
    elif x.split()[-1] in first2full:
        return x.split()[-1]
    else:
        return x

lines['Player'] = lines['Player'].apply(lambda x: map_name(x))
lines.Player.unique()

array(['Dean Kremer', 'Adley Rutschman', 'Gunnar Henderson',
       'Anthony Santander', 'Ryan Mountcastle', 'Adam Frazier',
       "Ryan O'Hearn", 'Nathan Eovaldi', 'Corey Seager', 'Marcus Semien',
       'Adolis Garcia', 'Josh Jung', 'Mitch Garver', 'Bryce Elder',
       'Ronald Acuna', 'Matt Olson', 'Austin Riley', 'Ozzie Albies',
       'Marcell Ozuna', 'Eddie Rosario', 'Aaron Nola', 'Bryce Harper',
       'Trea Turner', 'Kyle Schwarber', 'Nick Castellanos',
       'J.T. Realmuto', 'Alec Bohm', 'Bryson Stott', 'Johan Rojas',
       'Lance Lynn', 'Mookie Betts', 'Freddie Freeman', 'Will Smith',
       'Max Muncy', 'J.D. Martinez', 'Jason Heyward', 'David Peralta',
       'Miguel Rojas', 'Brandon Pfaadt', 'Corbin Carroll', 'Ketel Marte',
       'Christian Walker', 'Tommy Pham', 'Gabriel Moreno',
       'Evan Longoria', 'Geraldo Perdomo', 'Lourdes Gurriel Jr.',
       'Alek Thomas', 'Filip Forsberg', 'Juuse Saros', 'Roman Josi',
       'Thomas Novak', "Ryan O'Reilly", 'Tyson Barrie', 

In [18]:
league = []
for p in guys:
    if p in p2l:
        league.append(p2l[p])
    elif p.split()[-1] in p2l:
        league.append(p2l[p.split()[-1]])
    else:
        league.append('other')
league[0:5]

['MLB', 'MLB', 'MLB', 'MLB', 'MLB']

In [19]:
guys[1000:1007]

['Kaapo Kahkonen',
 'Sergei Bobrovsky',
 'Connor Hellebuyck',
 'Petr Mrazek',
 'Adin Hill',
 'Darcy Kuemper',
 'John Gibson']

In [20]:
team_id = []
for p in guys:
    if p in p2l:
        team_id.append(p2tm[p])
    elif p.split()[-1] in p2l:
        team_id.append(p2tm[p.split()[-1]])
    else:
        team_id.append('other')
team_id[0:5]

['1fd765d9-7354-4926-bb4b-368d86d202b0',
 '1fd765d9-7354-4926-bb4b-368d86d202b0',
 '1fd765d9-7354-4926-bb4b-368d86d202b0',
 '1fd765d9-7354-4926-bb4b-368d86d202b0',
 '1fd765d9-7354-4926-bb4b-368d86d202b0']

In [21]:
lines['League'] = league
lines

,Line,Stat,title,Player,League
0,3.5,strikeouts,Dean Kremer Strikeouts O/U,Dean Kremer,MLB
1,11.5,pitch_outs,Dean Kremer Pitching Outs O/U,Dean Kremer,MLB
2,2.0,runs_allowed,Dean Kremer Earned Runs Allowed O/U,Dean Kremer,MLB
3,1.5,walks_allowed,Dean Kremer Walks Allowed O/U,Dean Kremer,MLB
4,7.0,fantasy_points,Adley Rutschman Fantasy Points O/U,Adley Rutschman,MLB
...,...,...,...,...,...
1039,13.5,fantasy_points,J. Heim + E. Carter Fantasy Points O/U,J. Heim + E. Carter,other
1040,12.5,fantasy_points,A. Hays + R. O'Hearn Fantasy Points O/U,A. Hays + R. O'Hearn,other
1041,2.5,total_bases,A. Hays + R. O'Hearn Total Bases O/U,A. Hays + R. O'Hearn,other
1042,3.0,hits_runs_rbis,J. Heim + E. Carter Hits + Runs + RBIs O/U,J. Heim + E. Carter,other


In [22]:
def map_team(x):
    if x in p2tm:
        return p2tm[x]
    elif p.split()[-1] in p2l:
        return p2tm[p.split()[-1]]
    else:
        return x

lines['team_id'] = lines.Player.apply(lambda x: map_team(x))
lines

,Line,Stat,title,Player,League,team_id
0,3.5,strikeouts,Dean Kremer Strikeouts O/U,Dean Kremer,MLB,1fd765d9-7354-4926-bb4b-368d86d202b0
1,11.5,pitch_outs,Dean Kremer Pitching Outs O/U,Dean Kremer,MLB,1fd765d9-7354-4926-bb4b-368d86d202b0
2,2.0,runs_allowed,Dean Kremer Earned Runs Allowed O/U,Dean Kremer,MLB,1fd765d9-7354-4926-bb4b-368d86d202b0
3,1.5,walks_allowed,Dean Kremer Walks Allowed O/U,Dean Kremer,MLB,1fd765d9-7354-4926-bb4b-368d86d202b0
4,7.0,fantasy_points,Adley Rutschman Fantasy Points O/U,Adley Rutschman,MLB,1fd765d9-7354-4926-bb4b-368d86d202b0
...,...,...,...,...,...,...
1039,13.5,fantasy_points,J. Heim + E. Carter Fantasy Points O/U,J. Heim + E. Carter,other,976becd7-ee88-4db5-9ae6-d1bb6b2b16a8
1040,12.5,fantasy_points,A. Hays + R. O'Hearn Fantasy Points O/U,A. Hays + R. O'Hearn,other,a2ddbb25-4d77-4b86-9b27-27e150eede67
1041,2.5,total_bases,A. Hays + R. O'Hearn Total Bases O/U,A. Hays + R. O'Hearn,other,a2ddbb25-4d77-4b86-9b27-27e150eede67
1042,3.0,hits_runs_rbis,J. Heim + E. Carter Hits + Runs + RBIs O/U,J. Heim + E. Carter,other,976becd7-ee88-4db5-9ae6-d1bb6b2b16a8


In [23]:
def map_id(x):
    if x in id2tm:
        return id2tm[x]
    else:
        return x
#lines['team_id'] = lines.team_id.apply(lambda x: map_id(x))


In [24]:
final = pd.merge(lines,matchup, how='left',left_on='team_id',right_on='away_id')
final = final.drop(['Home', 'Away','home_id','away_id'], axis=1)
final2 = pd.merge(final,matchup, how='left',left_on='team_id',right_on='home_id')
final2 = final2.drop(['Home', 'Away','home_id','away_id'], axis=1)
final2['Date'] = final2['Date_x'].fillna(final2['Date_y'])
final2 = final2.drop(['Date_x', 'Date_y'], axis=1)
final2['team_id'] = final2.team_id.apply(lambda x: map_id(x))
final2#.loc[final.Player=='Jimmy Butler']

,Line,Stat,title,Player,League,team_id,Date
0,3.5,strikeouts,Dean Kremer Strikeouts O/U,Dean Kremer,MLB,BAL,2023-10-10
1,11.5,pitch_outs,Dean Kremer Pitching Outs O/U,Dean Kremer,MLB,BAL,2023-10-10
2,2.0,runs_allowed,Dean Kremer Earned Runs Allowed O/U,Dean Kremer,MLB,BAL,2023-10-10
3,1.5,walks_allowed,Dean Kremer Walks Allowed O/U,Dean Kremer,MLB,BAL,2023-10-10
4,7.0,fantasy_points,Adley Rutschman Fantasy Points O/U,Adley Rutschman,MLB,BAL,2023-10-10
...,...,...,...,...,...,...,...
1039,13.5,fantasy_points,J. Heim + E. Carter Fantasy Points O/U,J. Heim + E. Carter,other,976becd7-ee88-4db5-9ae6-d1bb6b2b16a8,NaN
1040,12.5,fantasy_points,A. Hays + R. O'Hearn Fantasy Points O/U,A. Hays + R. O'Hearn,other,a2ddbb25-4d77-4b86-9b27-27e150eede67,NaN
1041,2.5,total_bases,A. Hays + R. O'Hearn Total Bases O/U,A. Hays + R. O'Hearn,other,a2ddbb25-4d77-4b86-9b27-27e150eede67,NaN
1042,3.0,hits_runs_rbis,J. Heim + E. Carter Hits + Runs + RBIs O/U,J. Heim + E. Carter,other,976becd7-ee88-4db5-9ae6-d1bb6b2b16a8,NaN


In [25]:
final2.columns = ['Line', 'Stat', 'title', 'Player', 'League', 'Team', 'Date']

In [26]:
df = final2[['Line', 'Stat', 'Player', 'League', 'Team', 'Date']].copy(deep=True)
df

,Line,Stat,Player,League,Team,Date
0,3.5,strikeouts,Dean Kremer,MLB,BAL,2023-10-10
1,11.5,pitch_outs,Dean Kremer,MLB,BAL,2023-10-10
2,2.0,runs_allowed,Dean Kremer,MLB,BAL,2023-10-10
3,1.5,walks_allowed,Dean Kremer,MLB,BAL,2023-10-10
4,7.0,fantasy_points,Adley Rutschman,MLB,BAL,2023-10-10
...,...,...,...,...,...,...
1039,13.5,fantasy_points,J. Heim + E. Carter,other,976becd7-ee88-4db5-9ae6-d1bb6b2b16a8,NaN
1040,12.5,fantasy_points,A. Hays + R. O'Hearn,other,a2ddbb25-4d77-4b86-9b27-27e150eede67,NaN
1041,2.5,total_bases,A. Hays + R. O'Hearn,other,a2ddbb25-4d77-4b86-9b27-27e150eede67,NaN
1042,3.0,hits_runs_rbis,J. Heim + E. Carter,other,976becd7-ee88-4db5-9ae6-d1bb6b2b16a8,NaN


In [27]:
df = df.reset_index(drop=True)

In [28]:
df.Stat.unique()

array(['strikeouts', 'pitch_outs', 'runs_allowed', 'walks_allowed',
       'fantasy_points', 'hits_runs_rbis', 'singles', 'runs',
       'total_bases', 'batter_strikeouts', 'hits_allowed', 'walks',
       'points', 'goals_against', 'saves', 'shots', 'assists',
       'power_play_points', 'passing_yds', 'rush_rec_tds', 'rushing_yds',
       'receiving_yds', 'passing_tds', 'passing_ints', 'receiving_rec',
       'extra_points_made', 'kicking_points', 'passing_and_rushing_yds',
       'rush_rec_yds', 'rebounds', 'pts_rebs_asts', 'passing_att',
       'passing_comps', 'tackles', 'tackles_and_assists', 'rushing_att',
       'field_goals_made', 'sacks', 'aces', 'double_faults',
       'games_played', 'games_lost', 'breakpoints_won', 'games_won',
       'sets_lost', 'service_games_lost', 'strokes', 'birdies_or_better',
       'bogeys_or_worse', 'cumulative_par_3_strokes',
       'fantasy_points_on_maps_1_2', 'kills_in_maps_1_2',
       'map_1_and_2_kills', 'headshots_on_maps_1_2', 'position',

In [29]:
syntax = {
    'pts_rebs_asts': 'PTS+TRB+AST',
    'pts_rebs': 'PTS+TRB',
    'points': 'PTS',
    'free_throws_made': 'FT',
    'fantasy_points': 'Fantasy',
    'rebounds': 'TRB',
    'assists': 'AST',
    'steals': 'STL',
    'blocks': 'BLK',
    'blks_stls': 'BLK+STL',
    'turnovers': 'TO',
    'rebs_asts': 'TRB+AST',
    'pts_asts': 'PTS+AST',
    'three_points_made': '3PM',
    'shots': 'Shots',
    'goals': 'Goals',
    'saves': 'Goalie Saves',
    'clearances': 'Clearances',
    'goals_and_assists': 'Goals and Assists',
    'tackles': 'Tackles',
    'interceptions': 'Interceptions',
    'crosses': 'Crosses',
    'goals_against': 'Goal Against',
    'game_1_and_2_kills': 'MAPS 1-2 Kills',
    'map_1_and_2_kills': 'MAPS 1-2 Kills',
    'Headshots on Maps 1+2': 'MAPS 1-2 Headshots',
    'strokes': 'Strokes',
    'bogeys_or_worse': 'Bogeys or Worse',
    'birdies_or_better': 'Birdies or Better',
    'aces': 'Aces',
    'breakpoints_won': 'Break Points Won',
    'games_won': 'Games Won',
    'games_lost': 'Games Lost',
    'total_games': 'Total Games',
    'double_faults': 'Double Faults',
    'service_games_lost': 'Service Games Lost',
    'Fastest Laps': 'Fastest Laps',
    'position': 'position',
    'passing_yds': 'Pass Yards',
    'rushing_yds': 'Rush Yards',
    'receiving_yds': 'Receiving Yards' ,
    'fantasy_points': 'Hitter Fantasy Score',
    'strikeouts': 'Pitcher Strikeouts',
    'hits_runs_rbis': 'PTS',
    'singles':'Singles',
    'total_bases': 'Total Bases',
    'walks': 'Walks Allowed',
    'hits': 'Hits Allowed',
                }

In [30]:
df = df.replace(syntax)

In [31]:
df.loc[df['League'] == 'ESPORTS']

,Line,Stat,Player,League,Team,Date
713,29.3,fantasy_points_on_maps_1_2,ApaMen,ESPORTS,Detonation FocusMe,2023-10-11
714,3.0,kills_in_maps_1_2,ApaMen,ESPORTS,Detonation FocusMe,2023-10-11
715,13.0,kills_in_maps_1_2,Aria+Yutapon,ESPORTS,Detonation FocusMe,2023-10-11
716,40.7,fantasy_points_on_maps_1_2,Steal,ESPORTS,Detonation FocusMe,2023-10-11
717,3.0,kills_in_maps_1_2,Steal,ESPORTS,Detonation FocusMe,2023-10-11
...,...,...,...,...,...,...
851,38.5,MAPS 1-2 Kills,Bruninho,ESPORTS,FURIA Academy,2023-10-10
852,17.5,headshots_on_maps_1_2,lub,ESPORTS,FURIA Academy,2023-10-10
853,12.5,headshots_on_maps_1_2,mello,ESPORTS,FURIA Academy,2023-10-10
854,22.0,headshots_on_maps_1_2,kye,ESPORTS,FURIA Academy,2023-10-10


In [32]:
df.loc[df.Player=='Krimbo']

,Line,Stat,Player,League,Team,Date


In [33]:
today = dt.date.today()

In [34]:
str(today)

'2023-10-10'

In [38]:
df.columns = [x.lower() for x in df.columns]
df.to_csv(f'Lines/doglines{today.month}_{today.day}.csv')

In [37]:
df.loc[df.league=='ESPORTS']

,line,stat,player,league,team,date
713,29.3,fantasy_points_on_maps_1_2,ApaMen,ESPORTS,Detonation FocusMe,2023-10-11
714,3.0,kills_in_maps_1_2,ApaMen,ESPORTS,Detonation FocusMe,2023-10-11
715,13.0,kills_in_maps_1_2,Aria+Yutapon,ESPORTS,Detonation FocusMe,2023-10-11
716,40.7,fantasy_points_on_maps_1_2,Steal,ESPORTS,Detonation FocusMe,2023-10-11
717,3.0,kills_in_maps_1_2,Steal,ESPORTS,Detonation FocusMe,2023-10-11
...,...,...,...,...,...,...
851,38.5,MAPS 1-2 Kills,Bruninho,ESPORTS,FURIA Academy,2023-10-10
852,17.5,headshots_on_maps_1_2,lub,ESPORTS,FURIA Academy,2023-10-10
853,12.5,headshots_on_maps_1_2,mello,ESPORTS,FURIA Academy,2023-10-10
854,22.0,headshots_on_maps_1_2,kye,ESPORTS,FURIA Academy,2023-10-10
